**Introduction**

Most PDF to text parsers do not provide layout information. Often times, even the sentences are split with arbritrary CR/LFs making it very difficult to find paragraph boundaries. This poses various challenges in chunking and adding long running contextual information such as section header to the passages while indexing/vectorizing PDFs for LLM applications such as retrieval augmented generation (RAG).

LayoutPDFReader solves this problem by parsing PDFs along with hierarchical layout information such as:

Sections and subsections along with their levels.
Paragraphs - combines lines.
Links between sections and paragraphs.
Tables along with the section the tables are found in.
Lists and nested lists.
With LayoutPDFReader, developers can find optimal chunks of text to vectorize, and a solution for limited context window sizes of LLMs.

# New Section

**Installation**

Install the llmsherpa library.

In [26]:
# !pip install llmsherpa

The first step in using the LayoutPDFReader is to provide a url or file path to it and get back a document object.

In [27]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "https://arxiv.org/pdf/1910.13461.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdfpdf_url = "https://arxiv.org/pdf/1910.13461.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# pdf_url = "https://www.census.gov/hfp/btos/downloads/CES-WP-24-16.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf

# pdf_url = "https://arxiv.org/pdf/2212.14024.pdf"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

**Install LlamaIndex**

In the following examples, we will use LlamaIndex for simplicity. Install the library if you haven't already.

In [28]:
# !pip install llama-index

**Setup OpenAI**

Make sure your API Key is inserted.

In [29]:
import openai
import os
from getpass import getpass
# from openai import OpenAI

if os.environ.get('OAI_KEY'):
    print(f"got OAI Key from environment var: 'OIA_KEY'")
    oai_key = os.environ['OAI_KEY']
else:
    oai_key = getpass("Enter an OPENAI key: ")
    os.environ['OAI_KEY'] = oai_key
oai_client = openai.OpenAI(api_key=oai_key)
# oai_key

got OAI Key from environment var: 'OIA_KEY'


**Summarize a Section using prompts**

LayoutPDFReader offers powerful ways to pick sections and subsections from a large document and use LLMs to extract insights from a section.

The following code looks for the Fine-tuning section of the document:

In [30]:
from IPython.core.display import display, HTML
selected_section = None
# find a section in the document by title
for section in doc.sections():
    if section.title == '3 Fine-tuning BART':
        #if '3. AI Use Rates' in section.title:
        print(f"SELECTED Section title: {section.title} -- subsections:{len(section.sections())}")
        selected_section = section
    else: 
        print(f"Section title: {section.title}")

Section title: BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension
Section title: {mikelewis,yinhanliu,naman}@fb.com
Section title: Abstract
Section title: 1 Introduction
Section title: B D A B C D E
Section title: Bidirectional Encoder
Section title: A _ C _ E
Section title: <s> A B C D
Section title: A B C D E
Section title: 2 Model
Section title: 2.1 Architecture
Section title: 2.2 Pre-training BART
Section title: Token Deletion Text Inﬁlling
SELECTED Section title: 3 Fine-tuning BART -- subsections:4
Section title: 3.1 Sequence Classiﬁcation Tasks
Section title: 3.2 Token Classiﬁcation Tasks
Section title: 3.3 Sequence Generation Tasks
Section title: 3.4 Machine Translation
Section title: 4 Comparing Pre-training Objectives
Section title: 4.1 Comparison Objectives
Section title: label
Section title: A B C D E <s> A B C D E
Section title: 4.2 Tasks
Section title: 4.3 Results
Section title: 5 Large-scale Pre-training Exper

/tmp/ipykernel_705983/909753822.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [31]:
for s in selected_section.sections():
    print(f"Section: {section.title} ")

Section: References 
Section: References 
Section: References 
Section: References 


In [32]:
# use include_children=True and recurse=True to fully expand the section.
# include_children only returns at one sublevel of children whereas recurse goes through all the descendants
#HTML(section.to_html(include_children=True, recurse=True))
HTML(selected_section.to_html(include_children=True, recurse=True))

Now, let's create a custom summary of this text using a prompt:

In [33]:
# from llama_index.llms import OpenAI
from llama_index.llms.openai import OpenAI
context = selected_section.to_html(include_children=True, recurse=True)
question = "list all the tasks discussed and one line about each task"
resp = OpenAI().complete(f"read this text and answer question: {question}:\n{context}")
print(resp.text)

Tasks discussed in the text:
1. Sequence Classification Tasks: Involves feeding the same input into the encoder and decoder, and using the final hidden state of the final decoder token for classification.
2. Token Classification Tasks: Involves feeding the complete document into the encoder and decoder, using the top hidden state of the decoder as a representation for each word for classification.
3. Sequence Generation Tasks: Involves fine-tuning BART for tasks like abstractive question answering and summarization, where the encoder input is the input sequence and the decoder generates outputs autoregressively.
4. Machine Translation: Involves using the entire BART model as a single pretrained decoder for machine translation by adding a new set of encoder parameters learned from bitext.


In [34]:
# from llama_index.llms import OpenAI
# from llama_index.llms.openai import OpenAI
context = selected_section.to_html(include_children=True, recurse=True)
# question = ""

resp = openai.Completion.create(
            engine="davinci",
            prompt=(f"Please summarize the following text:\n{context}\n\nSummary:"),
            temperature=0.5,
            max_tokens=1024,
            n = 1,
            stop=None
        )
resp

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
# resp = OpenAI().complete(f"read this text and answer question: {question}:\n{context}")
print(resp.text)

**Analyze a Table using prompts**

With LayoutPDFReader, you can iterate through all the tables in a document and use the power of LLMs to analyze a Table Let's look at the 6th table in this document. If you are using a notebook, you can display the table as follows:

In [ ]:
from IPython.core.display import display, HTML
HTML(doc.tables()[5].to_html())

Now let's ask a question to analyze this table:

In [ ]:
# from llama_index.llms import OpenAI
from llama_index.llms.openai import OpenAI
context = doc.tables()[5].to_html()
resp = OpenAI().complete(f"read this table and answer question: which model has the best performance on squad 2.0:\n{context}")
print(resp.text)

That's it! LayoutPDFReader also supports tables with nested headers and header rows.

Here's an example with nested headers (note that the HTML doesn't render properly in ipython but the html structure is correct):

In [ ]:
from IPython.core.display import display, HTML
HTML(doc.tables()[6].to_html())

Now let's ask an interesting question:

In [ ]:
from llama_index.llms.openai import OpenAI
context = doc.tables()[6].to_html()
question = "tell me about R1 of bart for different datasets"
resp = OpenAI().complete(f"read this table and answer question: {question}:\n{context}")
print(resp.text)


**Vector search and Retrieval Augmented Generation with Smart Chunking**

LayoutPDFReader does smart chunking keeping the integrity of related text together:

All list items are together including the paragraph that precedes the list.
Items in a table are chuncked together
Contextual information from section headers and nested section headers is included
The following code creates a LlamaIndex query engine from LayoutPDFReader document chunks

In [ ]:
from llama_index.core import Document
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex([])
for chunk in doc.chunks():
    index.insert(Document(text=chunk.to_context_text(), extra_info={}))
query_engine = index.as_query_engine()

Let's run one query:

In [ ]:
response = query_engine.query("list all the tasks that work with bart")
print(response)

Let's try another query that needs answer from a table:

In [ ]:
response = query_engine.query("what is the bart performance score on squad")
print(response)

**Get the Raw JSON**

To get the complete json returned by llmsherpa service and process it differently, simply get the json attribute

In [ ]:
doc.json